<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Only do this once per VM, otherwise you'll get multiple clones and nested directories
!git clone https://github.com/jamestheengineer/data-science-from-scratch-Python.git
%cd data-science-from-scratch-Python/
!pip install import-ipynb
import import_ipynb

In [ ]:
# Deep learning chapter

Tensor = list

from typing import List

def shape(tensor: Tensor) -> List[int]:
  sizes: List[int] = []
  while isinstance(tensor, list):
    sizes.append(len(tensor))
    tensor = tensor[0]
  return sizes

assert shape([1, 2, 3]) == [3]
assert shape([[1, 2], [3, 4], [5, 6]]) == [3, 2]


In [ ]:
def is_1d(tensor: Tensor) -> bool:
  """
  If tensor[0] is a list, it's a higher-order tensor.
  Otherwise, tensor is 1-dimensional (that is, a vector)
  """
  return not isinstance(tensor[0], list)

assert is_1d([1,2,3])
assert not is_1d([[1,2],[3,4]])

In [ ]:
def tensor_sum(tensor: Tensor) -> float:
  """Sums up all the values in the tensor"""
  if is_1d(tensor):
    return sum(tensor) # just a list of floats, use Python sum
  else:
    return sum(tensor_sum(tensor_i) # Call tensor_sum on each row
               for tensor_i in tensor) # and sum up those results

assert tensor_sum([1,2,3]) == 6
assert tensor_sum([[1,2],[3,4]]) == 10

In [ ]:
from typing import Callable

def tensor_apply(f: Callable[[float], float], tensor: Tensor) -> Tensor:
  """Applies f elementwise"""
  if is_1d(tensor):
    return [f(x) for x in tensor]
  else:
    return [tensor_apply(f, tensor_i) for tensor_i in tensor]

assert tensor_apply(lambda x: x+1, [1, 2, 3]) == [2,3,4]
assert tensor_apply(lambda x: 2 * x, [[1,2],[3,4]]) == [[2,4],[6,8]]

In [ ]:
def zeros_like(tensor: Tensor) -> Tensor:
  return tensor_apply(lambda _: 0.0, tensor)

assert zeros_like([1,2,3]) == [0,0,0]
assert zeros_like([[1,2],[3,4]]) == [[0,0],[0,0]]

In [ ]:
def tensor_combine(f: Callable[[float, float], float],
                   t1: Tensor,
                   t2: Tensor) -> Tensor:
    """Applies f to corresponding elements of t1 and t2"""
    if is_1d(t1):
      return [f(x,y) for x, y in zip(t1,t2)]
    else:
      return [tensor_combine(f, t1_i, t2_i)
              for t1_i, t2_i in zip(t1, t2)]

import operator

assert tensor_combine(operator.add, [1,2,3],[4,5,6]) == [5,7,9]
assert tensor_combine(operator.mul, [1,2,3], [4,5,6]) == [4,10,18]

In [ ]:
from typing import Iterable, Tuple

class Layer:
  """
  Our neural networks will be composed of layers, each of which
  knows how to do some computation on its inputs in the "forward"
  direction and propagate gradients in the "backward" direction.
  """
  def backward(self, gradient):
    """
    Similarly, we're not going to be prescriptive about what the
    gradient looks like. It's up to you the user to make sure 
    that you're doing things sensibly.
    """
    raise NotImplementedError
  
  def params(self) -> Iterable[Tensor]:
    """
    Returns the parameters of this layer. The default implementation 
    returns nothing, so that if you have a layer with no parameters
    you don't have to implement this.
    """
    return ()

  def grads(self) -> Iterable[Tensor]:
    """
    Returns the gradients, in the smae order as params().
    """
    return ()

In [ ]:
from Chapter_18 import sigmoid

class Sigmoid(Layer):
  def forward(self, input: Tensor) -> Tensor:
    """
    Apply sigmoid to each element of the input tensor,
    and save the results to use in backpropagation.
    """
    self.sigmoids = tensor_apply(sigmoid, input)
    return self.sigmoids

  def backward(self, gradient: Tensor) -> Tensor:
    return tensor_combine(lambda sig, grad: sig * (1 - sig) * grad,
                          self.sigmoids,
                          gradient)

In [11]:
# Functions to randomly generate our weight tensors
import random

from Chapter_06 import inverse_normal_cdf

def random_uniform(*dims: int) -> Tensor:
  if len(dims) == 1:
    return [random.random() for _ in range(dims[0])]
  else:
    return [random_uniform(*dims[1:]) for _ in range(dims[0])]

def random_normal(*dims: int,
                  mean: float = 1.0,
                  variance: float = 1.0) -> Tensor:
  if len(dims) == 1:
    return [mean + variance * inverse_normal_cdf(random.random())
            for _ in range(dims[0])]
  else:
    return [random_normal(*dims[1:], mean=mean, variance=variance)
            for _ in range(dims[0])]

assert shape(random_uniform(2,3,4)) == [2,3,4]
assert shape(random_normal(5,6,mean=10)) == [5,6]

In [12]:
# Wrap them all in a random_tensor function

def random_tensor(*dims: int, init: str = 'normal') -> Tensor:
  if init == 'normal':
    return random_normal(*dims)
  elif init == 'uniform':
    return random_uniform(*dims)
  elif init == 'xavier':
    variance = len(dims) / sum(dims)
    return random_normal(*dims, variance=variance)
  else:
    raise ValueError(f"unknown init: {init}")